In [69]:
from tools.torch_lib import *

In [70]:
### uncomment if libs are not installed:
# %pip install numpy
# %pip install pandas
# pytorch without gpu support:
# %pip install torch torchvision torchaudio
## tf without gpu support
# pip install tensorflow
# pip install keras
# pip install onnx
# pip install onnx2keras

In [71]:
# use only scripted (.pt) models!
models_dir = "models/"
print("enter model file name:")
model_name = input()
model_path = models_dir + model_name
scripted_model = torch.jit.load(model_path)
scripted_model.eval()

enter model file name:


RecursiveScriptModule(
  original_name=LinearModel
  (linears): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=Linear)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=Linear)
    (4): RecursiveScriptModule(original_name=Linear)
    (5): RecursiveScriptModule(original_name=Linear)
  )
  (activations): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=LeakyReLU)
    (1): RecursiveScriptModule(original_name=LeakyReLU)
    (2): RecursiveScriptModule(original_name=LeakyReLU)
    (3): RecursiveScriptModule(original_name=LeakyReLU)
    (4): RecursiveScriptModule(original_name=LeakyReLU)
    (5): RecursiveScriptModule(original_name=LeakyReLU)
  )
)

### convert to onnx

In [72]:
print(scripted_model)

RecursiveScriptModule(
  original_name=LinearModel
  (linears): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=Linear)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=Linear)
    (4): RecursiveScriptModule(original_name=Linear)
    (5): RecursiveScriptModule(original_name=Linear)
  )
  (activations): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=LeakyReLU)
    (1): RecursiveScriptModule(original_name=LeakyReLU)
    (2): RecursiveScriptModule(original_name=LeakyReLU)
    (3): RecursiveScriptModule(original_name=LeakyReLU)
    (4): RecursiveScriptModule(original_name=LeakyReLU)
    (5): RecursiveScriptModule(original_name=LeakyReLU)
  )
)


In [73]:
in_features = -1

for name, weights in scripted_model.named_parameters():
    shape = list(weights.size())

    if in_features == -1:
        in_features = shape[1]
    print(name, "shape=" + str(shape))

linears.0.weight shape=[50, 5]
linears.0.bias shape=[50]
linears.1.weight shape=[120, 50]
linears.1.bias shape=[120]
linears.2.weight shape=[1200, 120]
linears.2.bias shape=[1200]
linears.3.weight shape=[120, 1200]
linears.3.bias shape=[120]
linears.4.weight shape=[10, 120]
linears.4.bias shape=[10]
linears.5.weight shape=[1, 10]
linears.5.bias shape=[1]


In [74]:
# check inference
input_tensor = torch.randn(1, in_features)
scripted_model(input_tensor)

tensor([[1.3731]], grad_fn=<LeakyReluBackward0>)

In [75]:
input_tensor

tensor([[ 2.1839,  0.0583,  0.8178,  1.4903, -1.6197]])

In [76]:
torch.onnx.export(scripted_model, input_tensor, models_dir + "model.onnx", export_params=True, opset_version=11)
#onnx_binary.save(models_dir + "onnx_model.onnx")

C:\GitHub\NSU_Graduate\PtKerasConversion\venv\Lib\site-packages\torch\onnx\utils.py:847: UserWarning: no signature found for <torch.ScriptMethod object at 0x000001BBEA8D6270>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


### Convert onnx model to keras

In [77]:
import onnx
from onnx2keras import onnx_to_keras

onnx_model = onnx.load(models_dir + "model.onnx")
input_all = [node.name for node in onnx_model.graph.input]
keras_model = onnx_to_keras(onnx_model, input_all, name_policy='renumerate')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> x.1.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> result1.19.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight linears.0.weight with shape (50, 5).
DEBUG:onnx2keras:Found weight linears.0.bias with shape (50,).
DEBUG:onnx2keras:Found weight linears.1.weight with shape (120, 50).
DEBUG:onnx2keras:Found weight linears.1.bias with shape (120,).
DEBUG:onnx2keras:Found weight linears.2.weight with shape (1200, 120).
DEBUG:onnx2keras:Found weight linears.2.bias with shape (1200,).
DEBUG:onnx2keras:Found weight linears.3.weight with shape (120, 1200).
DEBUG:onnx2keras:Found weight linears.3.bias with shape (120,).
DEBUG:onnx2keras:Found weight linears.4.weight with shape (10, 120).
DEBUG:onnx2keras:Found weight linears.4.bias with shape (10,).
DEBUG:onnx2keras:Found weight linears.5.weight with sh

In [78]:
keras_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x.1 (InputLayer)            [(None, 5)]               0         
                                                                 
 LAYER_0 (Dense)             (None, 50)                300       
                                                                 
 LAYER_1 (LeakyReLU)         (None, 50)                0         
                                                                 
 LAYER_2 (Dense)             (None, 120)               6120      
                                                                 
 LAYER_3 (LeakyReLU)         (None, 120)               0         
                                                                 
 LAYER_4 (Dense)             (None, 1200)              145200    
                                                                 
 LAYER_5 (LeakyReLU)         (None, 1200)              0   

In [79]:
keras_model.predict(input_tensor.numpy())

1/1 [==============================] - 0s 73ms/step


array([[1.3731353]], dtype=float32)

In [80]:
# save keras model
keras_model.save(models_dir + "keras_model.h5")

C:\GitHub\NSU_Graduate\PtKerasConversion\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [81]:
from keras.models import load_model
# check loaded keras model
keras_model = load_model(models_dir + "keras_model.h5")
keras_model.predict(input_tensor.numpy())

1/1 [==============================] - 0s 83ms/step


array([[1.3731353]], dtype=float32)

### convert model to keras2cpp lib format and save it
keras2cpp lib doesn't support Input() layer so convert model to contain only
Dense & Activation layers

In [82]:
from keras.src.layers import LeakyReLU
from keras import Sequential
from keras.layers import Dense, Activation

input_dim = in_features
converted_model = Sequential()

for layer in keras_model.layers[1:]:
    if isinstance(layer, Dense):
        insert_dense_layer = Dense(units=layer.units, input_shape=(input_dim,))
        converted_model.add(insert_dense_layer)
        converted_model.layers[-1].set_weights(layer.get_weights())
        input_dim = (layer.units, )
    else:
        converted_model.add(layer)  # activations


converted_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 50)                300       
                                                                 
 LAYER_1 (LeakyReLU)         (None, 50)                0         
                                                                 
 dense_20 (Dense)            (None, 120)               6120      
                                                                 
 LAYER_3 (LeakyReLU)         (None, 120)               0         
                                                                 
 dense_21 (Dense)            (None, 1200)              145200    
                                                                 
 LAYER_5 (LeakyReLU)         (None, 1200)              0         
                                                                 
 dense_22 (Dense)            (None, 120)              

In [83]:
# check converted model inference
converted_model.predict(input_tensor.numpy())

1/1 [==============================] - 0s 70ms/step


array([[1.3731353]], dtype=float32)

In [84]:
from tools.keras2cpp import export_model
export_model(converted_model, models_dir + model_name + '.model')